In [34]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC

In [35]:
data = pd.read_csv('winequality-red.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [36]:
# Обучение модели случайного леса

X = data.drop(['quality'], axis = 1)
y = data['quality']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

rand_forest_classif = RandomForestClassifier()
rand_forest_classif.fit(X_train, y_train)

y_pred = rand_forest_classif.predict(X_test)
print(classification_report(y_test, y_pred))

# Важность признаков для модели классификации (Gini impurity)
feats = {}
for feature, importance in zip(data.columns, rand_forest_classif.feature_importances_):
    feats[feature] = importance

importances = pd.DataFrame.from_dict(feats, orient='index').rename(columns={0: 'Gini-Importance'})
print(importances.sort_values(by='Gini-Importance', ascending=False))

              precision    recall  f1-score   support

           0       0.78      0.76      0.77       213
           1       0.81      0.83      0.82       267

    accuracy                           0.80       480
   macro avg       0.80      0.79      0.79       480
weighted avg       0.80      0.80      0.80       480

                      Gini-Importance
alcohol                      0.190253
sulphates                    0.133746
volatile acidity             0.107636
total sulfur dioxide         0.100088
density                      0.093044
chlorides                    0.073374
citric acid                  0.062625
fixed acidity                0.062466
pH                           0.062209
free sulfur dioxide          0.057463
residual sugar               0.057095


Если связывать качество вина с его стоимостью, то важнее определять вино плохого качества, которое позиционируется как вино хорошего качества. Необходимо снизить количество ложноположительных объектов, поэтому больше подходит метрика точность (precision). Если учесть цену ошибки, то важна точность для вина истинно низкого качества (precision 0). 

Самый информативный признак - содержание спирта (alcohol). Наименее информативный - остаточный сахар (residual sugar).

In [37]:
# Сравнение с методом опорных векторов

svm = SVC(kernel ='linear')
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.66      0.74      0.70       213
           1       0.77      0.70      0.73       267

    accuracy                           0.72       480
   macro avg       0.72      0.72      0.72       480
weighted avg       0.72      0.72      0.72       480



Вывод: для  линейной SVM-модели выбранная метрика оценки качества имеет меньшее значение (precision для ложноположительных объектов (0) 0.66 против 0.78 для RFC). Следовательно, модель случайного леса справляется с задачей лучше.